In [1]:
import pandas as pd
import os
from ast import literal_eval
import numpy as np
import glob
from pathlib import Path
import multiprocessing


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/chrisliao/.conda/envs/oss_hierarchy/lib/python3.12/runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/chrisliao/.conda/envs/oss_hierarchy/lib/python3.12/runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "/home/chrisliao/.conda/envs/oss_hierarchy/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/chrisliao/.local/lib/python3.12/site-packages/traitlets/config/application.py"

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/chrisliao/.conda/envs/oss_hierarchy/lib/python3.12/runpy.py", line 198, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/chrisliao/.conda/envs/oss_hierarchy/lib/python3.12/runpy.py", line 88, in _run_code
    exec(code, run_globals)
  File "/home/chrisliao/.conda/envs/oss_hierarchy/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/chrisliao/.local/lib/python3.12/site-packages/traitlets/config/application.py"

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



In [2]:
os.chdir('../')

In [16]:
def ReadPrCommits(filename):
    df_pr_commits = pd.read_csv(filename, index_col = 0).drop(
        'Unnamed: 0', axis = 1)
    if 'commit_list' in df_pr_commits.columns:
        df_pr_commits['commit_list'] = df_pr_commits['commit_list'].apply(
            lambda x: literal_eval(x) if x != "'float' object has no attribute 'split'" else np.nan)
        return df_pr_commits.dropna()

def ReadPushCommits(filename):
    df_push_commits = pd.read_csv(filename, index_col = 0)
    df_push_commits['commit_urls_parsed'] = df_push_commits['commit_urls'].apply(literal_eval)
    df_push_commits['commit_urls_parsed'] = df_push_commits['commit_urls_parsed'].apply(
        lambda x: [ele if ele.startswith("https:") else "https:"+ele for ele in x[0].split("https:")][1:] if len(x) == 1 else x)
    df_push_commits['commit_list'] = df_push_commits['commit_urls_parsed'].apply(lambda x: [ele.split("/")[-1] for ele in x])
    df_push_commits['repo_name'] = df_push_commits['commit_urls_parsed'].apply(lambda x: "/".join(x[0].split("/")[4:6]) if len(x)>0 else np.nan)

    return df_push_commits

def SelectFirst(x):
    return x.loc[x.index[0]]

def GetUniquePushes(repo, df_push_commits, df_pr_commits, outdir):
    df_push_commits_repo = df_push_commits.query(f'repo_name == "{repo}"').explode('commit_list')
    df_pr_commits_repo = df_pr_commits.query(f'repo_name == "{repo}"').explode('commit_list')


    df_push_commits_merged = pd.merge(df_push_commits_repo, df_pr_commits_repo, 
                                      how = 'left', on = ['repo_name','commit_list'])
    df_push_commits_merged = df_push_commits_merged[df_push_commits_merged['pr_commits_url'].isna()]
    df_push_commits_uq = df_push_commits_merged.groupby(
        ['repo_name','push_id']).agg({'commit_urls_parsed': SelectFirst, 'commit_list': list}).reset_index()

    print(repo, df_push_commits_uq.shape)
    df_push_commits_uq.to_csv(outdir / f'unique_push_{repo.replace("/","___")}')

def PoolWorker(repo):
    return GetUniquePushes(repo, df_push_commits, df_pr_commits, outdir)

In [8]:
outdir = Path('drive/output/derived/filter_nonunique_push')
df_pr_commits = pd.concat([ReadPrCommits(filename) for filename in glob.glob('drive/output/scrape/push_pr_commit_data/pull_request_data/*')])
df_push_commits = pd.concat([ReadPushCommits(filename) for filename in glob.glob('drive/output/scrape/push_pr_commit_data/push_data/*')])

df_push_commits.reset_index(inplace = True, drop = True)
df_pr_commits.reset_index(inplace = True, drop = True)

In [18]:
repo_list = df_push_commits['repo_name'].dropna().unique().tolist()

with multiprocessing.Pool(processes=8) as pool:
    results = pool.map(PoolWorker, repo_list)

KristianOellegaard/django-hvad (75, 4)
nickoala/telepot (341, 4)
sanand0/xmljson (52, 4)
ros-infrastructure/rosinstall_generator (70, 4)
sdispater/orator (302, 4)
bartTC/pip-check (49, 4)
py4n6/pytsk (180, 4)
angr/angr (4949, 4)
hannorein/rebound (2548, 4)
schlamar/cov-core (1, 4)
pyca/bcrypt (461, 4)
trentm/python-markdown2 (269, 4)
nltk/nltk (949, 4)
kk7ds/somecomfort (38, 4)
pysal/pysal (182, 4)
michael-lazar/rtv (698, 4)
nabla-c0d3/sslyze (701, 4)
spookylukey/django-paypal (203, 4)
ome/omego (48, 4)
python/mypy (6131, 4)
nicolargo/glances (1978, 4)
nvbn/django-bower (22, 4)
borntyping/python-colorlog (123, 4)
VUnit/vunit (1594, 4)
Mottie/tablesorter (854, 4)
xattr/xattr (109, 4)
peterbe/django-fancy-cache (57, 4)
fopina/pyspeedtest (32, 4)
IDSIA/sacred (529, 4)
Z3Prover/z3 (8624, 4)
kinverarity1/lasio (312, 4)
kvesteri/intervals (46, 4)
konstantint/matplotlib-venn (16, 4)
santoshphilip/eppy (592, 4)
mailgun/talon (102, 4)
jpadilla/django-rest-framework-oauth (13, 4)
aws/aws-cli (35